In [ ]:
import spacy
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.spatial.distance import pdist, squareform
from scipy.special import kl_div
from scipy.stats import entropy  # provides Shannon entropy and KL divergence
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
#from sklearn.feature_extraction.text import TfidfVectorizer
#import networkx as nx
from nltk import *
import math
import numpy as np    
import copy






libraries:
spacy: tokenize corpus, stopwords,lemmazation, text cleaning, pos tagging
requests: access dracor and opera-guide.ch
beautifulsoup: html sparsing for getting main text and metadata from opera guide
pandas: data storing and manupilation
collections, counter: count words and frequencies
plt: visualisation
seaborn: visualisation
scipy: entropy/kl_div: kl divergence; pdist,quareform:metric distance
sklearn:pca,cosine_similarity
nltk: generate n grams
math
numpy
copy: dictionary munplation

In [ ]:
#initialize corpus from dracor outside the function to aviod repeat loading

from pydracor import  Corpus
sh_corpus = Corpus("gersh")
ger_corpus = Corpus("ger")
ger_meta = ger_corpus.metadata()
sh_meta = sh_corpus.metadata()
print(ger_meta)

In [ ]:
def DCText(corpus_name, filter_kwargs=None):
    from pydracor import DraCor, Corpus

    if corpus_name == "gersh":
        corpus = sh_corpus
        metadata = sh_meta
    else:
        corpus = ger_corpus 
        metadata = ger_meta


    
    
    if filter_kwargs:
        corpus = corpus.filter(**filter_kwargs) #corpus became list with play ids
        #print(corpus)
    else:
        corpus = list(corpus.play_ids())

    data={}
    names,authors,years,ids,spoken_text=[], [], [], [], []

    for i in range(len(corpus)):
        
        #print(i)
        play_meta = next(item for item in metadata if item['id'] == corpus[i])

        if int(play_meta['word_count_text']) < 7400:
            continue

        author_count = len(set(authors))
        if author_count == 0:
            author_count = 1
        if  author_count > 5 and len(names) == 25:  # stop if we already have 5 authors or 5*5 plays
            break 

        author = play_meta['first_author']


        if authors.count(author) >= 10: #each author has maxium 5 plays
            continue
        

        
        authors.append(play_meta['first_author'])
        years.append(play_meta['year_normalized'])
        ids.append(play_meta['id'])
        names.append(play_meta['name'])

    # use correct corpus in the URL
        url = f"https://dracor.org/api/v1/corpora/{corpus_name}/plays/{play_meta['name']}/spoken-text"
        response = requests.get(url)
        spoken_text.append(response.text)
        #print(f"{names} ({corpus_name})\n", spoken_text[i][:30])  # Print preview

    data["names"] =names
    data["authors"]=authors
    data["ids"]=ids
    data["years"]=years
    data["spoken_text"]=spoken_text

    print(len(set(data['authors'])))
    print(len(set(data['names'])))
    return data


In [ ]:
titles = frozenset([
    "Romeo und Julia",
    "Der Sturm",
    "Hamlet. Prinz von Dänemark",
    "Maß für Maß",
    "König Lear",
    "Macbeth",
    "Ein Sommernachtstraum",
    "Othello",
    "Wie es euch gefällt",
    "König Johann"
])

sh_data = DCText("gersh",{'title__in':titles})

wa_data=DCText("ger",{"authors__name__icontains": "Wagner, Richard"})

print(sh_data["names"])

fuction1: get text from dracor through pydracor and dracor api
saves all the plays that fit the conditionaly filter in one dictionary: data ={names:[play names to be used later in dracor api],authors:[author names],years:[normolised years],ids:[play ids],spoken_text=[]}
time effeciency: access meta data of the whole german corpus and get spoken text through pydracor takes too much times. solutions to that are first: move the lines of accessing the metadata metadata = corpus.metadata()
out from the funtion so it will only be pulled once; use dracor api to get the spoken text which is way faster beacuse it pulls the text directly through website. doing so saves plenty of runtime in testing and final proformence. to access the desired plays through api requires the play names. playnames are saved in metadata from pycor. thus we can realise the final funtion through a intergration: query the metadata through pydracor and get the list of target playnames, then get the sopken test using dracor api.


In [ ]:
# wagner's other works
urls = ["https://opera-guide.ch/operas/das+liebesverbot/libretto/de/",
      "https://opera-guide.ch/operas/die+feen/libretto/de/",
      "https://opera-guide.ch/operas/rienzi+der+letzte+der+tribunen/libretto/de/"]



def getHTMLSpokenText(url):
    # get play name from url
    name=re.search(r"/operas/([^/]+)/libretto/", url).group(1).replace("+", "-")

    response = requests.get(url)
    response.raise_for_status()



    soup = BeautifulSoup(response.text, "html.parser")

    # Step 1: Extract the div
    div = soup.find("div", class_="col-lg-7 col-12")
    if not div:
        return ""

    # Step 2: Make a clean copy of the div before modifing
    div_clean = BeautifulSoup(str(div), "html.parser").div

    # Step 3: Remove <i> tags
    for i_tag in div_clean.find_all("i"):
        i_tag.decompose()

    # Step 4: Get the raw text (before removing uppercase lines)
    full_text = div_clean.get_text(separator="\n")


    # Step 5: Trim everything before "ERSTER AKT"
    match = re.search(
    r"\b(?:ERSTER?|I\.)\s+(?:AKT|AUFZUG|SZENE)\b", 
    full_text, 
    re.IGNORECASE
        )
    if match:
        full_text = full_text[match.start():]
    else:
        print("Warning: 'ERSTER AKT' not found.")
        return full_text.strip()  # return full uncut version just in case

    # Step 6: Remove all-uppercase lines
    lines = full_text.splitlines()
    clean_lines = [line for line in lines if not line.strip().isupper() and line.strip() != ""]
    text = "\n".join(clean_lines)

    return text.strip(),name

2.get adiitional plays from opera guide website:
3 plays from wagner which are not predent in Dracor are listed in another website. By html prasing through beautiful soup we can access the texts. 

after close inspection, we can notice that all texts are stored in <div, class_="col-lg-7 col-12"> tags. 
apart from the spoken text we need, there are:
1.there areadditional information before actual text starts(names, summaries), text starts with'FIRST AKT', or'I. Aufzug/SEZNE' in different plays' cases.
2.all the character names are in cap and spereate lines with the spoken text. 
3.stage directions below charater names in <i> tags.

solution:
2. use .decompose() from bf4 to remove all stage direction
1. find"first akt" as a marker through regex and get the main text throuh .start() from bf4 (.decompose() tags before "first akt" element will cause critical problem that destories the whole tree and the function will return "" as result)
3.use regex to find all cap charcters(only names in this case) then remove them.

addtionally to align with previous DCText() function we gather the play names from the url directly at the same time. the result of the meta data can be included manually most effciently.



In [ ]:

#merging additional plays to wagner corpus

for url in urls:
    add_text,add_name = getHTMLSpokenText(url)
    wa_data['spoken_text'].append(add_text)
    wa_data["names"].append(add_name)
    wa_data["authors"].append('Wagner')
    wa_data['ids'].append(str('opera_guide_'+str(urls.index(url))))
wa_data['years'].extend([1835,1888,1842])
#print( wa_data['years'])

In [ ]:

def combineData(*datasets):

    #combines multiple dict datasets with the same keys

    if not datasets:
        return {}
    
    combined = {}
    for key in datasets[0]:
        combined[key] = []
        for data in datasets:
            combined[key] += data[key]
    return combined


In [ ]:
#get control sets fromt the same era
control = frozenset([
    "Schiller",
    "Grillparzer",
    "Hebbel",
])
co_data={'names':[],'authors':[],'ids':[],'years':[],'spoken_text':[]}
for name in control:
    #print(name)
    data= DCText("ger",{"authors__name__icontains":name})
    #print(data)
    co_data=combineData(data,co_data)


In [ ]:
jon={'names':[],'authors':[],'ids':[],'years':[],'spoken_text':[]}
jon= DCText("ger",{"name__contains":"schlegel-jon"})
co_data=combineData(jon,co_data)
#print(co_data)

In [ ]:

# tokenize all texts in courpus dictionary once and store
def tokenizeData(data, nlp):
    data["tokens"] = [nlp(text) for text in data["spoken_text"]]
    return data

# tokenize all datasets
nlp = spacy.load("de_core_news_sm")

wa_data = tokenizeData(wa_data, nlp)
sh_data = tokenizeData(sh_data, nlp)
co_data =  tokenizeData(co_data,nlp)

In [ ]:
# get and store POS Tag in dictionary

def getPOS(tokens):
    return [token.pos_ for token in tokens if token.is_alpha] #if token.is_alpha?

def dataPOS(data):
    pos_tags=[]
    for tokens in data['tokens']:
        posed=getPOS(tokens)
        pos_tags.append(posed)

    data['pos_tags']= pos_tags
    return data


co_data = dataPOS(co_data)
wa_data =dataPOS(wa_data)
sh_data =dataPOS(sh_data)
#print(co_data['pos_tags'])


In [ ]:
combined_data=combineData(wa_data,sh_data,co_data)
#print(combined_data)

In [ ]:

# get min token length for normalisation
def findMin(data):
    return min(len(doc) for doc in data["tokens"])

min_tokens = findMin(combined_data)
print(f"Shortest corpus length (in tokens): {min_tokens}")
min_tokens = findMin(wa_data)
print(f"Shortest corpus length (in tokens): {min_tokens}")
min_tokens = findMin(sh_data)
print(f"Shortest corpus length (in tokens): {min_tokens}")

normoalise corpus lenth:Findmin() find out minium lenth in the entrie corpus. all the plays will be trimed into same lenth to aviod bias when nesseary.

function words kl divergence:
1.get all word count from every play/ every author
2.normalized them so that the components summed to 1
2.took each of these normalized vectors to be the feature vector for the corresponding author/play
4.apply kl then similarity to dataframe.
5.visuisation in heatmap

In [ ]:
# Function Word Frequencies
def getFunWord(text):
    return str(" ".join([token.lemma_ for token in text[:min_tokens] if token.is_stop]).lower()).split()


map out all the function words in a play, stored them as list , each word is one element, and only the function words with in the minlen will be gathered.
spacy doesnt have a fix stopword list.

In [ ]:

def corpusFreq(data):
    corpus_count={}
    for name, tokens in zip(data["names"], data["tokens"]):
        fun_Words =getFunWord(tokens)
        counts = Counter(fun_Words)
        print(name,counts)

        corpus_count[name]=counts
    return corpus_count
corpus_count= corpusFreq(combined_data)
print(corpus_count)


corpus_freq()returns a nested dictionary, key is play name and value will be a collection of funtion words and thier counts in value pairs. the countering and storing the values are easily done by counter() from collections.

In [ ]:


df_count = pd.DataFrame.from_dict(corpus_count, orient='index').fillna(0) #Builds  count matrix, row = play, column = a word/feature.
#KL involves log(P/Q). If P>0 and Q=0, KL explodes to ∞. Smoothing prevents division by zero and infinite values. but if .fillna(ep), all the counts will be 1 eps short.
df_count['authors']=combined_data['authors']
df_avg=df_count.groupby(['authors']).sum()
eps=1e-8
df_avg += eps
df_prob = df_avg.div(df_avg.sum(axis=1), axis=0)

#L1-normalize to make probabilities divide by the sum so the vector sums to 1


print(df_prob)

next setp will be the first data stastical method on funtionword frequency. kl direvence .
first prepare the data in data frame. Builds  count matrix, row = play, column = a word/feature. we wan to see the performance on author level, so we group all the texts by author name and sum up all the word counts from single plays. 
#KL involves log(P/Q). If P>0 and Q=0, KL explodes to ∞. Smoothing prevents division by zero and infinite values. but if .fillna(ep), all the counts will be 1 eps short.
second, different author has diffrenet total number of plays and to calculate kl , datas must sum up to 1. 
so Division makes each row sum to 1 → normalized probability distribution of words per author.

In [ ]:

def kl(df):
    index = df.index
    kl_matrix = pd.DataFrame(index=index, columns=index, dtype=float)
    for a1 in index:
        for a2 in index:
            p = df.loc[a1]
            q = df.loc[a2]
            kl_matrix.loc[a1, a2] = kl_div(p, q).sum() # KL(a1 || a2)
    return kl_matrix

kl_matrix = kl(df_prob)
print(kl_matrix)


KL with spicy:
https://stackoverflow.com/questions/63369974/3-functions-for-computing-relative-entropy-in-scipy-whats-the-difference

In [ ]:
def sim(df):
    sigma = 0.5
    similarity_matrix = np.exp(-df / sigma)

   
    similarity_matrix = pd.DataFrame(similarity_matrix, index=df.index, columns=df.columns)
    return similarity_matrix

similarity_matrix = sim(kl_matrix)
print(similarity_matrix)


# heatmap visulisation:
#print(df_kl)
plt.figure(figsize=(8, 8))
sns.heatmap(similarity_matrix, annot=True, cmap='viridis_r', fmt=".2f", linewidths=.5,vmin=0.5, vmax=1)
plt.title('Similarity Matrix based on KL Divergence')
plt.xlabel('Author')
plt.ylabel('Author')
plt.tight_layout()
plt.show()


convert kl values to similarities according to Hughes et al.
the value will be in [0,1], the higher the more similar.

In [ ]:
#kl divergence of each plays
df =df_count.select_dtypes(include=['number'])
df += eps
df_all_prob = df.div(df.sum(axis=1), axis=0)

#print(df_all_prob)


kl_matrix_all = kl(df_all_prob)
#print(kl_matrix)

similarity_matrix_all= sim(kl_matrix_all)

plt.figure(figsize=(80, 80))
sns.heatmap(similarity_matrix_all, annot=True, cmap='viridis_r', fmt=".2f", linewidths=.5,vmin=0, vmax=1)
plt.title('Similarity Matrix based on KL Divergence')
plt.xlabel('plays')
plt.ylabel('palys')
plt.tight_layout()
plt.show()





pos uni gram:</br>
1.extract all pos for each play</br>
2.group df by author and calulate the means for each pos tag frequency under same author</br>
3.calculate proportion of each tags for each author in dataframe</br>
4.stacked bar plot per auhtor</br>

In [ ]:
#Feature Extraction: POS Tag Frequencies and N-grams

#function to calculate normalized POS tag frequencies (unigrams)
def calculate_pos_frequencies(pos):
    pos_count = Counter(pos)
    total_tags = sum(pos_count.values())
    # Normalize to get proportions/percentages
    pos_freq = {tag: count / total_tags for tag, count in pos_count.items()}
    return pos_freq

# function to calculate normalized POS tag N-gram frequencies
def calculate_pos_ngrams(pos, n):
    pos_ngrams = list(ngrams(pos, n))
    ngram_count = Counter(pos_ngrams)
    total_ngrams = sum(ngram_count.values())
    # Normalize to get proportions/probabilities
    ngram_freq = {ngram: count / total_ngrams for ngram, count in ngram_count.items()}
    return ngram_freq

# store calculated features for each play
pos_frequencies = {}
pos_bigrams = {}
pos_trigrams = {}

for i, name in enumerate(combined_data["names"]):
    pos_tags = combined_data["pos_tags"][i]
    pos_frequencies[name] = calculate_pos_frequencies(pos_tags)
    pos_bigrams[name] = calculate_pos_ngrams(pos_tags, n=2)
    pos_trigrams[name] = calculate_pos_ngrams(pos_tags, n=3)


In [ ]:

# convert  frequency dictionary to a DataFrame
tag_df = pd.DataFrame.from_dict(pos_frequencies, orient="index").fillna(0.0)
tag_df['authors'] = combined_data['authors']


author_means = tag_df.groupby("authors").mean().reset_index()
# drop non-POS columns first
pos_columns = [col for col in author_means.columns if col not in ['authors']]

# calculate row-wise percentage for each author
author_means_percentage = author_means.copy()
author_means_percentage[pos_columns] = author_means_percentage[pos_columns].div(
    author_means_percentage[pos_columns].sum(axis=1), axis=0
) * 100  # multiply by 100 to get percentage

#print(author_means_percentage)




pos_columns = [col for col in author_means_percentage.columns if col != 'authors']
df = author_means_percentage.set_index('authors')[pos_columns]

fig, ax = plt.subplots(figsize=(14, 6))

# stacked bar plot
bars = df.plot(kind='bar', stacked=True, ax=ax, colormap='tab20')

# annotate each segment with percentage
for i, author in enumerate(df.index):
    bottom = 0
    for pos in pos_columns:
        val = df.loc[author, pos]
        if val > 0:
            ax.text(
                i, 
                bottom + val/2,           # position in the middle of the segment
                f"{val:.1f}%",            # show one decimal
                ha='center', va='center',
                fontsize=8,
                color='white'         
            )
            bottom += val

plt.ylabel("Percentage of POS Tags")
plt.title("POS Tag Distribution per Author")
plt.xticks(rotation=45)
plt.legend(title="POS Tag", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()




POS bigram:</br>
1.plot a heat map for each play based on bigram frequency, the differencies are hard to spot</br>
2.calculate z-score to find out the distinguisble bi gram usage among all plays</br>
3.Compute cosine similarity to generate dendrogram</br>
4.Plot dendrogram</br>

In [ ]:
# Prepare data for heatmaps (bigram frequency matrices)
# create a heatmap for each author

all_unique_tags = sorted(list(set(tag for tags_list in combined_data["pos_tags"] for tag in tags_list)))

n_authors = len(combined_data["names"])
n_rows, n_cols = 6, 9  # 6x9 grid
fig, axes = plt.subplots(n_rows, n_cols, figsize=(4*n_cols, 4*n_rows))  # adjust size as needed

# flatten axes array for easy iteration
axes = axes.flatten()

for ax, name in zip(axes, combined_data["names"]):
    bigram_freqs = pos_bigrams[name]

    # create a matrix for the heatmap
    bigram_matrix = pd.DataFrame(0.0, index=all_unique_tags, columns=all_unique_tags)
    for (tag1, tag2), freq in bigram_freqs.items():
        if tag1 in all_unique_tags and tag2 in all_unique_tags:
            bigram_matrix.loc[tag1, tag2] = freq
    #print(bigram_matrix)
    
    # plot heatmap on the specific axis
    sns.heatmap(bigram_matrix, annot=False, fmt=".2f", cmap="Blues",
                linewidths=.5, linecolor='lightgray', cbar=True, ax=ax, vmin=0, vmax=0.01, center=0.003)
    ax.set_title(f'{name}', fontsize=10)
    ax.set_xlabel('Second Tag', fontsize=8)
    ax.set_ylabel('First Tag', fontsize=8)

# turn off any unused subplots
for i in range(len(combined_data["names"]), len(axes)):
    axes[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# bi gram z score

# gather all unique bigrams
import pandas as pd
import numpy as np
#print(pos_bigrams)
# Gather all unique bigrams and convert to a list
all_bigrams=[]
for text_name, bigram_counts in pos_bigrams.items():
    for bigram, freq in bigram_counts.items():
        # bigram is a tuple like ('AUX', 'PRON')
        bi_str = f"{bigram[0]}-{bigram[1]}"  # convert tuple to string
        if bi_str not in all_bigrams:
            all_bigrams.append(bi_str)
        #print(bi_str, freq)

#print(all_bigrams)
# create a DataFrame: rows=texts, columns=bigrams
df = pd.DataFrame(index=pos_bigrams.keys(), columns=all_bigrams).fillna(0.0)


for text, counts in pos_bigrams.items():
    for bigram, freq in counts.items():
        bi_str = f"{bigram[0]}-{bigram[1]}"  # convert tuple to string
        df.loc[text, bi_str] = freq

#print(df)
# compute z-scores for each bigram
z_scores = df.copy()
z_scores['authors']=combined_data['authors']
for col in df.columns:
    mean = df[col].mean()
    std = df[col].std(ddof=0)  # population std; use ddof=1 for sample std
    if std > 0:
        z_scores[col] = (df[col] - mean) / std
    else:
        z_scores[col] = 0.0  # if no variance, z-score is zero

print(z_scores)

In [ ]:
z_scores = z_scores.reset_index(drop=True)  # drop old index
z_scores['text'] = combined_data['names']   # create a proper text/play column
print(z_scores.columns)


In [ ]:

# select bigram columns
bigram_cols = [col for col in z_scores.columns if '-' in col]
X = z_scores[bigram_cols].fillna(0)

# compute cosine similarity
sim_matrix = cosine_similarity(X)

# convert to distance
dist_matrix = 1 - sim_matrix

# ensure diagonal is exactly zero
np.fill_diagonal(dist_matrix, 0)

# flatten for linkage
dist_condensed = squareform(dist_matrix)


# hierarchical clustering
Z = linkage(dist_condensed, method='ward')

# plot dendrogram
plt.figure(figsize=(14, 8))
dendro = dendrogram(
    Z,
    labels=z_scores['text'].values,
    leaf_rotation=90,
    leaf_font_size=10,
)

# color leaves by author
ax = plt.gca()
author_list = z_scores['authors'].tolist()
unique_authors = z_scores['authors'].unique()
colors = plt.cm.tab10(range(len(unique_authors)))
author_colors = dict(zip(unique_authors, colors))

for tick_label in ax.get_xticklabels():
    text = tick_label.get_text()
    author = z_scores.loc[z_scores['text'] == text, 'authors'].values[0]
    tick_label.set_color(author_colors[author])

plt.title("Dendrogram of texts based on POS bigram z-scores (cosine distance)")
plt.xlabel("Text")
plt.ylabel("Distance")
plt.show()


In [ ]:
'''
#setting a threshhold of 2 for z score, the result is the same

#print(X)
z_filtered = X.copy()
z_filtered = z_filtered.mask(z_filtered.abs() >= 2,0)


# Compute cosine similarity and distance
sim_matrix = cosine_similarity(z_filtered)
dist_matrix = 1 - sim_matrix

# Ensure diagonal is exactly zero
np.fill_diagonal(dist_matrix, 0)

# Flatten for linkage
dist_condensed = squareform(dist_matrix)


# Hierarchical clustering
Z = linkage(dist_condensed, method='ward')

# Plot dendrogram
plt.figure(figsize=(14, 8))
dendro = dendrogram(
    Z,
    labels=z_scores['text'].values,
    leaf_rotation=90,
    leaf_font_size=10,
)

# Color leaves by author
ax = plt.gca()
author_list = z_scores['authors'].tolist()
unique_authors = z_scores['authors'].unique()
colors = plt.cm.tab10(range(len(unique_authors)))
author_colors = dict(zip(unique_authors, colors))

for tick_label in ax.get_xticklabels():
    text = tick_label.get_text()
    author = z_scores.loc[z_scores['text'] == text, 'authors'].values[0]
    tick_label.set_color(author_colors[author])

plt.title("Dendrogram of texts based on POS bigram z-scores (cosine distance)")
plt.xlabel("Text")
plt.ylabel("Distance")
plt.show()
'''


In [ ]:
'''''
#PCA for cos_sim

# sim_matrix: cosine similarity matrix (numpy array)
# z_scores['text'] contains text names
# z_scores['authors'] contains authors

# Convert similarity to distance for PCA: distance = 1 - similarity
dist_matrix = 1 - sim_matrix

# Apply PCA
pca = PCA(n_components=2)
coords = pca.fit_transform(dist_matrix)

# Make a DataFrame for plotting
df_plot = pd.DataFrame({
    'PC1': coords[:, 0],
    'PC2': coords[:, 1],
    'text': z_scores['text'],
    'author': z_scores['authors']
})

# Scatter plot
plt.figure(figsize=(10, 8))
for author in df_plot['author'].unique():
    subset = df_plot[df_plot['author'] == author]
    plt.scatter(subset['PC1'], subset['PC2'], label=author)

# Add annotations (play names) for each point
for i, row in df_plot.iterrows():
    plt.text(row['PC1'] + 0.02, row['PC2'] + 0.02, row['text'], fontsize=8)  

plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("PCA of texts based on high z-score POS bigrams")
plt.legend()
plt.show()
'''


In [ ]:
##pca analysis on all grams

#combine POS uni-, bi-, and trigram frequencies
X = {}
for name in combined_data['names']:
    X[name] = copy.deepcopy(pos_frequencies[name])      # Start with unigrams
    X[name].update(pos_bigrams[name])                   # Add bigrams
    X[name].update(pos_trigrams[name])                  # Add trigrams

#normalize keys: turn tuples into strings
def string_key(k):
    if isinstance(k, str):
        return k
    return ">".join(k)  # Join POS tags with '>' to mark order

X_str_keys = {}
for name, feats in X.items():
    X_str_keys[name] = {string_key(k): v for k, v in feats.items()}

#build DataFrame
df_features = pd.DataFrame.from_dict(X_str_keys, orient="index").fillna(0)

print("Feature matrix shape:", df_features.shape)
print("Sample columns:", df_features.columns[:-10])

#apply PCA
pca = PCA(n_components=2) #collapse featrues into 2 dimensions for easy visualisation
principal_components = pca.fit_transform(df_features)

pca_df = pd.DataFrame(principal_components, columns=["PC1", "PC2"])
pca_df['authors']=combined_data['authors']
pca_df['years']=combined_data['years']
pca_df['names']=df_features.index


print("Explained variance ratio:", pca.explained_variance_ratio_)




In [ ]:

#Scatter plot
plt.figure(figsize=(15, 12))
sns.scatterplot(
    data=pca_df,
    x="PC1", y="PC2",
    hue="years",                # Color points by year , shape by author
    palette="viridis",
    style='authors',
    
    s=300
)

#add name labels for each point
for i, row in pca_df.iterrows():
    plt.text(
        row["PC1"] + 0.002,      # small horizontal offset
        row["PC2"] + 0.002,      # small vertical offset
        row["names"],            # column in pca_df with the play name
        fontsize=10,
        color="black"
    )


plt.title("PCA of POS N-gram Frequencies", fontsize=16)
plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% var)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% var)")
plt.axhline(0, color="grey", linewidth=0.5, linestyle="--")
plt.axvline(0, color="grey", linewidth=0.5, linestyle="--")
plt.legend(title="plays", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

